A good network has three attributes; it's routable, it's comparable to other networks and it's dynamic. A network's correctness is subjective and despite best efforts, even Google gets it wrong some times. However the better a network is georectified, the easier it will be to compare to other models. Otherwise, discrepancies should be noted and sameness should be highlighted. This sameness is confirmation that the networks are comparable and determining this sameness requires taking an automated look at both of the networks, because there are surely discrepancies. 

In [1]:
# this process will use arcpy to determine the sameness of two networks
import arcpy as ap

# the csv module is used at the end of this process to write tables to csv
import csv

# the os module is also used in this process, specifically to determine the project location,
import os


# the Colorado DOT's modeling network 
modeling_network_file = os.getcwd() + '\\Modeling_Network_only_roads_copy.shp'
    
# and the Open Street Map network, downloaded from GeoFabrik 
osm_network_file = os.getcwd() + '\\gis_osm_roads_free_1.shp'

The next cell is just used to set the directy for the output of files. If it doesn't already exist, this directory will be created in this cell.

In [2]:
# the project path is set to be the location of this file
path_of_project = os.getcwd()

# and the directory for data is a directory called Data in the project directory
path_to_data = path_of_project + '//Data//'

# if the directory for the data hasn't been created
if not os.path.exists(path_to_data):
    # it will be created
    os.makedirs(path_to_data)

# arcpy has a workplace parameter that sets the output to the data directory
ap.env.workspace = path_to_data

THe primary point of comparison of these models will be the lines' bearing, so in the following cell bearing will be added as an attribute to each network

In [3]:
for each in [modeling_network_file, osm_network_file]:
    try:
        ap.AddGeometryAttributes_management(each, 'LINE_BEARING')
    except:
        pass

For the most robust conflation, many points along the segments will be considered, specifically points every 50 Meters along the segments. 

In [ ]:
#try:
#    ap.Delete_management(path_to_data + 'modeling_network_points.shp')
#except:
#    pass

#ap.GeneratePointsAlongLines_management(modeling_network_file, 
#                                       path_to_data + 'modeling_network_points.shp', 
#                                       'DISTANCE', 
#                                       '50 Meters',
#                                       '', 
#                                       'NO_END_POINTS' )

The next step will be to consider the proximity of the points along lines to the open street maps network. The following cell simply creates a table that essentially describes the lines between the modeling network points and the open street map network. 

In [ ]:
try:
    ap.Delete_management(path_to_data + 'modeling_points_every_50M_near_osm_network.dbf')
except:
    pass


ap.GenerateNearTable_analysis(path_to_data + 'modeling_network_points.shp', 
                              osm_network_file,
                              path_to_data + 'modeling_points_every_50M_near_osm_network.dbf',
                              '75 Meters',
                              'LOCATION',
                              'ANGLE',
                              'ALL',
                               3,
                              'GEODESIC')

In [3]:
line_id_dict = {}

with ap.da.SearchCursor(path_to_data + 'modeling_network_points.shp', ['FID', 'ORIG_FID']) as sc:
    for row in sc:
        line_id_dict[row[0]] = row[1]

try:
    ap.AddField_management(path_to_data + 'modeling_points_every_50M_near_osm_network.dbf', 'LINE_ID', 'SHORT')
except:
    pass

with ap.da.UpdateCursor(path_to_data + 'modeling_points_every_50M_near_osm_network.dbf',
                       ['IN_FID', 'LINE_ID']) as uc:
    for row in uc:
        new_row = row
        new_row[1] = line_id_dict[row[0]]
        uc.updateRow(new_row)

Many segments will be nearly identical, these matches will be the first to be conflated and other less similiar segments will be compared lated. To create the simplest conflation a buffer of five Meters will be created around the modeling network. Five meters is slightly wider than a lane, making this distance more relevant. Matching segments from the open street maps network will be conflated to the corresponding matching segment in the modeling network.

In [4]:
import csv

try:
    os.remove(path_to_data + 'matches.csv')
except:
    pass

with open(path_to_data + 'matches.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    
    writer.writerow(['FID',
                     'modeling_id', 
                     'osm_id', 
                     'distance', 
                     'NEAR_RANK',
                     'NEAR_ANGLE', 
                     'FROM_X', 
                     'FROM_Y', 
                     'NEAR_X', 
                     'NEAR_Y'])
    
    
    with ap.da.SearchCursor(path_to_data + 'modeling_points_every_50M_near_osm_network.dbf', 
                             ['OID',
                             'LINE_ID', 
                             'NEAR_FID', 
                             'NEAR_DIST', 
                             'NEAR_RANK', 
                             'NEAR_ANGLE',
                             'FROM_X',
                             'FROM_Y',
                             'NEAR_X',
                             'NEAR_Y']) as sc:
        for row in sc:
            writer.writerow(row)

In [6]:
ap.Delete_management(path_to_data + 'all_match_lines.shp')

ap.XYToLine_management(path_to_data + 'matches.csv', 
                         path_to_data + 'all_match_lines.shp',
                         'from_x',
                         'from_y',
                         'near_x',
                         'near_y',
                         'GEODESIC',
                         'FID')

ap.AddGeometryAttributes_management(path_to_data + 'all_match_lines.shp', 'LENGTH_GEODESIC')

<Result 'C:\\Users\\mcintyrei\\Desktop\\Definitely\\OSM_Conflation//Data//all_match_lines.shp'>

In [7]:
try:
    os.remove(path_to_data + 'osm_attributes.csv')
except:
    pass

with open(path_to_data + 'osm_attributes.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['osm_id', 'osm_name', 'osm_type','osm_bearing'])
    
    with ap.da.SearchCursor (osm_network_file, ['FID', 'name', 'fclass', 'BEARING']) as sc:
        
        for row in sc:
            fid = row[0]
            name = row[1].encode('utf')
    
            fclass = row[2].encode('utf')
            bearing = round(row[3],2)
            
            writer.writerow([fid, name, fclass, bearing])

In [8]:
try:
    os.remove(path_to_data + 'modeling_attributes.csv')
except:
    pass

with open(path_to_data + 'modeling_attributes.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['modeling_id', 'modeling_name', 'modeling_type', 'modeling_bearing'])
    
    with ap.da.SearchCursor(modeling_network_file, ['FID', 'NAME', 'FACILITY_T', 'BEARING']) as sc:
        
        for row in sc:
            bearing = round(row[3],2)
            
            writer.writerow([row[0], row[1], row[2], bearing])

The next level of analysis will consider only the divided highways. It's simple to determine these segments based on their attributes, but this is not always accurate. However, this will not be addressed until we are done with thie initial analysis.